In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(5, 7), padding='same')(input_layer) # <- 기존 커널 7x7
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 3), padding='same')(x) # <- 기존 커널 3x3
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

# 유효한 라벨 정의
valid_labels = [1, 5, 9, 19, 23, 27, 37, 41, 45]

# 데이터 로드 및 전처리
def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

# 모델 훈련 및 평가
def train_and_evaluate(folder_path, csv_path, valid_labels, test_subjects):
    accuracies = []
    for test_subject in test_subjects:
        print(f"Testing on subject: {test_subject}")
        train_df, test_df = prepare_data(folder_path, csv_path, test_subject, valid_labels)
        
        train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
        train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
        train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
        train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
        
        test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
        test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
        test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
        test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
        
        right_eye_model = create_resnet18_model((128, 128, 3))
        left_eye_model = create_resnet18_model((128, 128, 3))
        mlp_model = create_mlp_model(train_features.shape[1:])
        combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
        x = layers.Dense(256, activation=mish)(combined_input)
        x = layers.Dropout(0.5)(x)

        # x = layers.Dense(128, activation=mish)(x)  # 새로운 Dense 계층 추가
        # x = layers.Dropout(0.4)(x)  # Dropout 추가
        
        # x = layers.Dense(64, activation=mish)(x)  # 또 다른 Dense 계층 추가
        # x = layers.Dropout(0.3)(x)  # Dropout 추가

        output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
        combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

        combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        history = combined_model.fit(
            [train_images_array, train_images_array, train_features], train_labels,
            validation_data=([test_images_array, test_images_array, test_features], test_labels),
            epochs=50,
            batch_size=1,
            verbose=1
        )
        
        predictions = combined_model.predict([test_images_array, test_images_array, test_features])
        predicted_classes = np.argmax(predictions, axis=1)
        actual_classes = np.argmax(test_labels, axis=1)

        loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
        accuracies.append(accuracy)
        print(f"Testing accuracy for subject {test_subject}: {accuracy:.4f}")
        
        # 결과 저장
        results = []
        for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
            test_df['Filename'], test_features, predicted_classes, actual_classes)):
            results.append({
                'Subject': test_subject,
                'Test Accuracy': accuracy,
                'Test Loss': loss,
                'Image File': image_path,
                'Feature': feature.tolist(),
                'Predicted Class': pred_class,
                'Actual Class': actual_class
            })
        
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(csv_path, f"0101_results_{test_subject}.csv"), index=False, encoding='utf-8')
        print(f"Results for subject {test_subject} saved.")
        
        # 모델 저장
        model_save_path = os.path.join(csv_path, f"0101_model_{test_subject}.h5")
        combined_model.save(model_save_path)
        print(f"Model saved at: {model_save_path}")
    
    print(f"Average Accuracy: {np.mean(accuracies):.4f}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
test_subjects = ['lgj', 'hsb', 'scy']  # 테스트로 사용할 대상

# LOSO 수행
train_and_evaluate(folder_path, csv_path, valid_labels, test_subjects)


Testing on subject: lgj
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_65', 'keras_tensor_130']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


1800/1800 ━━━━━━━━━━━━━━━━━━━━ 395s 211ms/step - accuracy: 0.2741 - loss: 2.2914 - val_accuracy: 0.3311 - val_loss: 2.0788
Epoch 2/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 379s 210ms/step - accuracy: 0.6841 - loss: 0.7737 - val_accuracy: 0.4644 - val_loss: 1.7177
Epoch 3/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 379s 211ms/step - accuracy: 0.8389 - loss: 0.3999 - val_accuracy: 0.5156 - val_loss: 1.8458
Epoch 4/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 379s 211ms/step - accuracy: 0.8861 - loss: 0.3080 - val_accuracy: 0.5533 - val_loss: 1.3481
Epoch 5/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 379s 211ms/step - accuracy: 0.9271 - loss: 0.2309 - val_accuracy: 0.2244 - val_loss: 6.9941
Epoch 6/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 380s 211ms/step - accuracy: 0.9266 - loss: 0.1876 - val_accuracy: 0.2800 - val_loss: 5.9088
Epoch 7/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 380s 211ms/step - accuracy: 0.9477 - loss: 0.1358 - val_accuracy: 0.3644 - val_loss: 3.6118
Epoch 8/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 380s 211ms/step - accuracy: 0.9

Testing accuracy for subject lgj: 0.3489
Results for subject lgj saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\0101_model_lgj.h5
Testing on subject: hsb
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_139', 'keras_tensor_204', 'keras_tensor_269']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


1800/1800 ━━━━━━━━━━━━━━━━━━━━ 409s 218ms/step - accuracy: 0.2016 - loss: 2.4532 - val_accuracy: 0.3000 - val_loss: 1.9524
Epoch 2/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 391s 217ms/step - accuracy: 0.6195 - loss: 0.8496 - val_accuracy: 0.3089 - val_loss: 2.2218
Epoch 3/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 391s 217ms/step - accuracy: 0.8495 - loss: 0.3854 - val_accuracy: 0.3800 - val_loss: 3.3398
Epoch 4/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 392s 218ms/step - accuracy: 0.8890 - loss: 0.2781 - val_accuracy: 0.3533 - val_loss: 4.3087
Epoch 5/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 391s 217ms/step - accuracy: 0.9361 - loss: 0.1847 - val_accuracy: 0.3711 - val_loss: 4.0411
Epoch 6/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 392s 218ms/step - accuracy: 0.9587 - loss: 0.1502 - val_accuracy: 0.2333 - val_loss: 6.0186
Epoch 7/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 393s 219ms/step - accuracy: 0.9386 - loss: 0.1744 - val_accuracy: 0.3511 - val_loss: 4.7769
Epoch 8/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 394s 219ms/step - accuracy: 0.9

Testing accuracy for subject hsb: 0.3889
Results for subject hsb saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\0101_model_hsb.h5
Testing on subject: scy
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_278', 'keras_tensor_343', 'keras_tensor_408']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


1800/1800 ━━━━━━━━━━━━━━━━━━━━ 432s 231ms/step - accuracy: 0.1962 - loss: 2.3845 - val_accuracy: 0.2778 - val_loss: 7.6384
Epoch 2/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 413s 229ms/step - accuracy: 0.6165 - loss: 0.9228 - val_accuracy: 0.2444 - val_loss: 12.2523
Epoch 3/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 413s 230ms/step - accuracy: 0.7803 - loss: 0.5271 - val_accuracy: 0.1622 - val_loss: 12.7291
Epoch 4/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 413s 230ms/step - accuracy: 0.8498 - loss: 0.3883 - val_accuracy: 0.1244 - val_loss: 24.5269
Epoch 5/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 414s 230ms/step - accuracy: 0.9033 - loss: 0.2759 - val_accuracy: 0.1978 - val_loss: 19.3180
Epoch 6/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 414s 230ms/step - accuracy: 0.9513 - loss: 0.1409 - val_accuracy: 0.3778 - val_loss: 5.2003
Epoch 7/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 413s 230ms/step - accuracy: 0.9142 - loss: 0.2390 - val_accuracy: 0.2311 - val_loss: 18.5990
Epoch 8/50
1800/1800 ━━━━━━━━━━━━━━━━━━━━ 415s 230ms/step - accuracy

Testing accuracy for subject scy: 0.1822
Results for subject scy saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\0101_model_scy.h5
Average Accuracy: 0.3067
